The first three sections below are a set of functions. The fourth section contains the main loop.

# 1. Importing Libraries and Initializing Parameters

NIFTY 100 Index is chosen for this assignment, and the list of its constituents are downloaded using [Nifty Indices](https://www.niftyindices.com/indices/equity/broad-based-indices/nifty-100) website.

In [4]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from datetime import date, timedelta
import math

import yfinance as yf

# 2. Data Downloading & Formatting


In [121]:
def get_daily_prices(price_dict):
    
    all_frames = []
    
    for symbol, df in price_dict.items():
        try:
            adj_close_series = df['Adj Close'].iloc[:, 0]
            
            monthly_df = adj_close_series.resample('ME').last().reset_index()
            
            monthly_df.columns = ['date', 'adj_cprice']
            monthly_df['symbol'] = symbol
            
            monthly_df['sym_date'] = (
                monthly_df['symbol'] + 
                "_" + 
                monthly_df['date'].dt.strftime('%Y-%m-%d')
            )
            
            all_frames.append(monthly_df)
            
        except KeyError:
            print(f"Warning: 'Adj Close' not found for {symbol}. Skipping.")
            continue
            
    if not all_frames:
        return pd.DataFrame()

    final_df = pd.concat(all_frames, ignore_index=True)
    return final_df[['date', 'adj_cprice', 'symbol', 'sym_date']]


In [5]:
def get_monthly_prices(pd_temp, last_date, symbol):
    #temp_pd_monthly = pd_temp[pd_temp.index <= pd.to_datetime(last_date)].resample('ME').last().reset_index()
    temp_pd_monthly = pd_temp[pd_temp.index <= pd.to_datetime(last_date)]
    temp_pd_monthly = temp_pd_monthly.resample('ME').last().reset_index()
    monthly_prices = pd.DataFrame()
    monthly_prices['date'] = temp_pd_monthly['Date']
    monthly_prices['adj_cprice'] = temp_pd_monthly['Adj Close']
    monthly_prices['symbol'] = symbol
    monthly_prices['sym_date'] = monthly_prices['symbol'] + '_' + monthly_prices['date'].astype(str)
    return monthly_prices


def get_weekly_prices(pd_temp, last_date, symbol):
    #temp_pd_weekly = pd_temp[pd_temp.index <= pd.to_datetime(last_date)].resample('W-Fri').last().reset_index()
    temp_pd_weekly = pd_temp[pd_temp.index <= pd.to_datetime(last_date)]
    temp_pd_weekly = temp_pd_weekly.resample('W-Fri').last().reset_index()
    weekly_prices = pd.DataFrame()
    weekly_prices['date'] = temp_pd_weekly['Date']
    weekly_prices['adj_cprice'] = temp_pd_weekly['Adj Close']
    weekly_prices['symbol'] = symbol
    weekly_prices['sym_date'] = weekly_prices['symbol'] + '_' + weekly_prices['date'].astype(str)
    return weekly_prices

In [89]:
def get_market_cap_series_from_master(symbol, master_price_store, start_date):

    px_raw = master_price_store.get(symbol)
    shares = MASTER_SHARES_STORE.get(symbol)

    if px_raw is None or px_raw.empty or shares is None or shares.empty:
        return None

    backup_period = start_date - pd.offsets.Day(30)
    end_date = start_date + pd.offsets.Day(1)

    px_raw = px_raw.loc[backup_period:end_date]

    if isinstance(px_raw.index, pd.MultiIndex):
        px_raw = px_raw.copy()
        px_raw.index = px_raw.index.get_level_values(0)

    if "Adj Close" in px_raw.columns:
        px = px_raw[["Adj Close"]].copy()
    elif "Close" in px_raw.columns:
        px = px_raw[["Close"]].copy()
    else:
        return None

    if isinstance(px.columns, pd.MultiIndex):
        px.columns = px.columns.get_level_values(0) 
    
    px = px.rename(columns={px.columns[0]: "price"})

    if px.empty:
        return None

    shares = shares.loc[:end_date]
    if isinstance(shares.index, pd.MultiIndex):
        shares = shares.copy()
        shares.index = shares.index.get_level_values(0)
    
    if isinstance(shares, pd.DataFrame):
        if isinstance(shares.columns, pd.MultiIndex):
            shares = shares.iloc[:, 0] 
    
    shares = shares.to_frame("shares")

    df = px.join(shares, how="left")

    df = px.join(shares, how="left")
    df["shares"] = df["shares"].ffill()

    valid_df = df.dropna(subset=["price", "shares"])
    if valid_df.empty:
        return None

    latest = valid_df.iloc[[-1]].copy()

    month_end = pd.to_datetime(start_date) + pd.offsets.MonthEnd(0)
    latest.index = [month_end]

    latest.reset_index(inplace=True)
    latest.rename(columns={"index": "date"}, inplace=True)

    latest["symbol"] = symbol
    latest["market_cap"] = latest["price"] * latest["shares"]
    latest["sym_date"] = symbol + "_" + latest["date"].astype(str)

    return latest.set_index("sym_date")


In [7]:
def build_price_and_mcap_data(universe_df, master_price_storage, init_constr_start_date, initial_construction_date):
    monthly_list = []
    weekly_list = []
    mcap_list = []

    for i in universe_df.index:
        symbol = universe_df.loc[i, "Symbol"]

        index_data = master_price_storage.get(symbol)
        if index_data is None or index_data.empty:
            continue

        # 3Y monthly history check
        check_pd = (
            index_data[index_data.index <= pd.to_datetime(initial_construction_date)]
            .resample("ME")
            .last()
        )

        if len(check_pd) < 36:
            continue

        # market cap snapshot
        mcap_ts = get_market_cap_series_from_master(
            symbol,
            master_price_storage,
            initial_construction_date
        )

        if mcap_ts is None:
            continue

        mcap_list.append(mcap_ts)

        monthly_list.append(
            get_monthly_prices(index_data, initial_construction_date, symbol)
        )

        weekly_list.append(
            get_weekly_prices(index_data, initial_construction_date, symbol)
        )

    if not mcap_list:
        raise RuntimeError("No valid securities")

    return (
        pd.concat(mcap_list),
        pd.concat(monthly_list).set_index("sym_date"),
        pd.concat(weekly_list).set_index("sym_date"),
    )


In [8]:
def build_master_price_storage(symbols, start_date, end_date):
    price_storage = {}

    for symbol in symbols:
        df = yf.download(
            f"{symbol}.NS",
            start=start_date,
            end=end_date,
            auto_adjust=False,
            progress=False
        )

        if df.empty:
            print(f"Skipping {symbol}: no data")
            continue

        df.index = pd.to_datetime(df.index)
        price_storage[symbol] = df

    return price_storage


In [9]:
def build_master_shares_storage(symbols):
    shares_storage = {}

    for symbol in symbols:
        tkr = yf.Ticker(f"{symbol}.NS")
        shares = tkr.get_shares_full()

        if shares is None or shares.empty:
            continue

        shares_storage[symbol] = (
            shares.tz_localize(None).sort_index()
        )

    return shares_storage


# 3. Momentum Strategy Development

For the strategy development, a static MIBOR rate of 6% is assumed throughout the calculations owing to data availability and time considerations.

## 3.1 Momentum Score (2.2)

In [10]:
def calculate_momentum_scores(target_date, symbols, monthly_df, weekly_df, is_conditional=False):

    results = []

    target_date = target_date + pd.offsets.MonthEnd(0)
    t_1 = target_date - DateOffset(months=1) + pd.offsets.MonthEnd(0)
    t_7 = target_date - DateOffset(months=7) + pd.offsets.MonthEnd(0)
    t_13 = target_date - DateOffset(months=13) + pd.offsets.MonthEnd(0)

    for sym in symbols:
        try:
            p_t_1 = monthly_df.loc[f"{sym}_{t_1.strftime('%Y-%m-%d')}", 'adj_cprice']
            p_t_7 = monthly_df.loc[f"{sym}_{t_7.strftime('%Y-%m-%d')}", 'adj_cprice']

            pm_6 = (p_t_1 / p_t_7) - 1 - 0.06 * 180 / 365

            pm_12 = np.nan
            if not is_conditional:
                p_t_13 = monthly_df.loc[f"{sym}_{t_13.strftime('%Y-%m-%d')}", 'adj_cprice']
                pm_12 = (p_t_1 / p_t_13) - 1 - 0.06

            mask = (weekly_df['symbol'] == sym) & (weekly_df['date'].dt.date <= t_1.date())

            temp_var = weekly_df[mask].reset_index().set_index('date').sort_index()
            weekly_price = temp_var.tail(36).reset_index().set_index('sym_date')
            
            ann_std_dev = weekly_price['adj_cprice'].pct_change().std() * np.sqrt(52)

            results.append({
                'symbol': sym,
                'ramv_6': pm_6/ann_std_dev,
                'ramv_12': pm_12/ann_std_dev if not is_conditional else np.nan,
            })

        except Exception:
            print('Shit went down!')
            continue # to skip stocks with missing data points
        
    df_scores = pd.DataFrame(results)

    # print(df_scores)

    df_scores['z_6'] = (df_scores['ramv_6'] - df_scores['ramv_6'].mean()) / df_scores['ramv_6'].std()
    
    if not is_conditional:
        df_scores['z_12'] = (df_scores['ramv_12'] - df_scores['ramv_12'].mean()) / df_scores['ramv_12'].std()
        df_scores['c'] = df_scores['z_6'] * 0.5 + df_scores['z_12'] * 0.5
    else:
        df_scores['c'] = df_scores['z_6'] * 0.5

    df_scores['z_unwin'] = (df_scores['c'] - df_scores['c'].mean()) / df_scores['c'].std()

    df_scores['z_win'] = df_scores['z_unwin'].clip(-3, 3)

    df_scores['momentum_score'] = np.where(
        df_scores['z_win'] > 0,
        1 + df_scores['z_win'],
        1 / (1 - df_scores['z_win'])
    )
    
    return df_scores

In [11]:
def calculate_momentum_weights(momentum_scores, market_caps):

    df = momentum_scores[['symbol', 'momentum_score']].merge(
        market_caps[['symbol', 'market_cap']],
        how='inner',
        on='symbol'
    )

    df['parent_weight'] = df['market_cap'] / df['market_cap'].sum()

    df['momentum_weight'] = df['parent_weight'] * df['momentum_score']

    df['momentum_weight'] = df['momentum_weight'] / df['momentum_weight'].sum()

    return df[['symbol', 'momentum_score', 'market_cap', 'parent_weight', 'momentum_weight']].sort_values('momentum_weight', ascending=False)


In [12]:
def initiate_momentum_calculation(target_date, monthly_df, weekly_df):
    symbols = monthly_df['symbol'].sort_values().unique()
    MIBOR_RATE = 0.06 # 6% per annum
    momentum_df = pd.DataFrame({
        'symbol': symbols,
        'date': target_date
    })

    momentum_df = momentum_df.loc[momentum_df['date']==target_date].merge(
        calculate_momentum_scores(target_date, symbols, monthly_df, weekly_df),
        how='inner',
        on='symbol'
    )

    momentum_df['date'] = pd.to_datetime(momentum_df['date'] + pd.offsets.MonthEnd(0)).dt.date
    momentum_df['sym_date'] = momentum_df['symbol'] + '_' + momentum_df['date'].astype(str)
    momentum_df.set_index('sym_date', inplace=True)

    return momentum_df

In [154]:
def calculate_volatility_threshold(index_data):
    index_data = index_data.sort_values('date')
    index_data['daily_ret'] = index_data['adj_cprice'].pct_change()
    
    index_data['vol'] = index_data['daily_ret'].rolling(window=63).std() * np.sqrt(250)
    
    monthly_vol = index_data.set_index('date')['vol'].resample('ME').last().dropna()
    
    delta = monthly_vol.pct_change().dropna()
    
    return delta.quantile(0.95)

In [155]:
def check_volatility_trigger(target_date, index_data, threshold):
    t_minus_1 = target_date - pd.offsets.MonthEnd(1)
    t_minus_2 = target_date - pd.offsets.MonthEnd(2)
    
    def get_vol_at(end_date):
        start_date = end_date - pd.offsets.MonthBegin(3)
        subset = index_data[(index_data['date'] >= start_date) & (index_data['date'] <= end_date)]
        if len(subset) < 40: # Minimum data requirement
            return None
        return subset['adj_cprice'].pct_change().std() * np.sqrt(250)

    vt = get_vol_at(t_minus_1)
    vt_minus_1 = get_vol_at(t_minus_2)
    
    if vt is None or vt_minus_1 is None:
        return False
        
    delta = (vt / vt_minus_1) - 1
    
    return delta > threshold

## 3.2 Securities Selection (2.3)

In [13]:
def round_off(numSec):
    if numSec < 100:
        return math.ceil(numSec/10)*10
    elif numSec >= 100 and numSec < 300:
        return math.ceil(numSec/25)*25
    else:
        return math.ceil(numSec/50)*50

In [14]:
def init_sec_algo(securities_df, NumSec):
    if NumSec <= 25:
        return securities_df
    
    else:
        target_mcap_20 = 0.2 * securities_df['market_cap'].sum()
        target_mcap_30 = 0.3 * securities_df['market_cap'].sum()

        securities_30pct_mcap = securities_df[securities_df['market_cap'].cumsum() <= target_mcap_30]
        
        NumSec_10pct = int (np.floor (0.1 * NumSec))
        NumSec_10pct_rounded = round_off(NumSec_10pct)
        
        NumSec_30pct_mcap = len(securities_30pct_mcap.index)
        NumSec_30pct_mcap_rounded = round_off(NumSec_30pct_mcap)

        NumSec_40pct = int(np.floor(0.4*NumSec))
        NumSec_40pct_rounded = round_off(NumSec_40pct)
        
        securities_40pct = securities_df.head(NumSec_40pct)

        if NumSec_30pct_mcap <= 25:
            return securities_df.head(round_off(25))
        
        elif NumSec_30pct_mcap > 25 and NumSec_30pct_mcap <= NumSec_10pct:
            return securities_df.head(NumSec_10pct_rounded)
        
        elif NumSec_30pct_mcap_rounded >= NumSec_40pct:
        
            if securities_40pct['market_cap'].sum() < target_mcap_20:
                return securities_40pct.head(NumSec_40pct_rounded)
        
            else:
                NumSec_20pct_mcap_rounded = round_off(securities_df['market_cap'].cumsum().searchsorted(target_mcap_20) + 1)
                return securities_df.head(NumSec_20pct_mcap_rounded)
        
        else:
            return securities_df.head(NumSec_30pct_mcap_rounded)



In [15]:
def SAIR_sec_algo(securities_df, NumSec, prevNumSec):
    target_mcap_10 = 0.1 * securities_df['market_cap'].sum()

    if prevNumSec > NumSec:
        return 1
    elif NumSec <= 25:
        return 0
    elif prevNumSec < 25:
        return 1
    elif securities_df.head(prevNumSec)['market_cap'].sum() <= target_mcap_10:
        return 1
    else:
        return -1

In [16]:
def buffer_rule(prev_sel_secs, cur_sel_secs, N):

    cur_sel_secs = cur_sel_secs.reset_index(drop=True)

    upper = int(np.floor(0.5 * N))
    lower = upper + N

    new_sel_secs = cur_sel_secs.iloc[: upper].copy()

    buffer_zone = cur_sel_secs.iloc[upper : lower]

    retained = buffer_zone[
        buffer_zone['symbol'].isin(prev_sel_secs['symbol'])
    ]

    new_sel_secs = pd.concat([
        new_sel_secs,
        retained
    ], ignore_index=True)

    sel_syms = set(new_sel_secs['symbol'])


    if len(new_sel_secs) < N:
        fillers = cur_sel_secs[
            ~cur_sel_secs['symbol'].isin(sel_syms)
        ]
        fillers = fillers.iloc[: (N - len(new_sel_secs))]
        new_sel_secs = pd.concat([
            new_sel_secs,
            fillers
        ], ignore_index=True)

    return new_sel_secs.iloc[:N]

In [17]:
def update_universe(previous_universe, current_parent_list):

    # Identifying Deletions (Stocks kicked out of Nifty 100)
    deletions = list(set(previous_universe) - set(current_parent_list))
    
    # Identifying Additions (New stocks entered Nifty 100)
    additions = list(set(current_parent_list) - set(previous_universe))
    
    if deletions:
        print(f"Parent Index Deletions: {deletions}")
    if additions:
        print(f"Parent Index Additions: {additions}")

    return current_parent_list

In [18]:
def determine_securities(target_date, momentum_df, market_cap_df, is_SAIR=True, prevNumSec=None):

    target_date = (target_date + pd.offsets.MonthEnd(0)).date()

    momentum_df = momentum_df.loc[momentum_df['date']==target_date]

    security_select_df = momentum_df[['symbol', 'date', 'z_unwin', 'momentum_score']].merge(
        market_cap_df[['symbol', 'market_cap']],
        how='inner',
        on='symbol'
        # left_index=True,
        # right_index=True,
    )

    security_select_df = security_select_df[security_select_df['market_cap'].notna()==True]
    
    ranked_securities = security_select_df.loc[security_select_df.rank().sort_values(['z_unwin', 'market_cap'], ascending=[False, True]).index]
    NumSec = len(ranked_securities.index)


    if is_SAIR:
        func_out = SAIR_sec_algo(ranked_securities, NumSec, prevNumSec)

        if func_out==1:
            return init_sec_algo(ranked_securities, NumSec)
        elif func_out==0:
            return ranked_securities
        elif func_out==-1:
            return ranked_securities.head(prevNumSec)


    else:
        return init_sec_algo(ranked_securities, NumSec)

# 4. MAIN LOOP

In [113]:
nifty100_companies = pd.read_csv('ind_nifty100list.csv')

# dates for initial construction
init_constr_start_date = date(2021, 9, 1)
init_constr_end_date = date(2024, 9, 1)


# date on which the initial construction of the momentum scores will be computed
initial_construction_date = date(2024, 9, 1)


# dates for conditional rebalancing and semi-annual index review, start_date is 4 months prior to initial_construction_date to consider data for V_t and V_t-1
start_date = date(2024, 5, 1) 
end_date = date(2026, 1, 1)

symbols = nifty100_companies["Symbol"].tolist()

MASTER_PRICE_STORE = build_master_price_storage(symbols, init_constr_start_date, end_date)

MASTER_DAILY_PRICES_pd = get_daily_prices(MASTER_PRICE_STORE)

MASTER_SHARES_STORE = build_master_shares_storage(symbols)

index_rebal_threshold = calculate_volatility_threshold(MASTER_DAILY_PRICES_pd)

$DUMMYHDLVR.NS: possibly delisted; no timezone found

1 Failed download:
['DUMMYHDLVR.NS']: possibly delisted; no timezone found


Skipping DUMMYHDLVR: no data


In [159]:
# INITIAL CONSTRUCTION - T_0

market_cap_data, monthly_data, weekly_data = build_price_and_mcap_data(nifty100_companies, MASTER_PRICE_STORE, init_constr_start_date, initial_construction_date)

momentum_data = initiate_momentum_calculation(initial_construction_date, monthly_data, weekly_data)

selected_securities = determine_securities(initial_construction_date, momentum_data, market_cap_data, False)

momentum_weights = calculate_momentum_weights(momentum_data, market_cap_data)

prevNumSec = curNumSec = len(selected_securities.index)

sim_start_date = (initial_construction_date + pd.offsets.MonthEnd(2)).date() # 2024-10-31
sim_end_date = end_date # 2025-12-31

sim_months = pd.date_range(sim_start_date, sim_end_date, freq='ME')

symbols = monthly_data['symbol'].sort_values().unique()

prev_month = (initial_construction_date + pd.offsets.MonthEnd(0)).date()

# T = 1, 2, 3, ..., end_time - moving forward in time to simulate and validate the momentum strategy

for simon in sim_months:

    # Ideally, old universe should be compared with new one, however, due to data availability and timing constraints, for this assignment, the only NIFTY 100 constituents initially acquired from the CSV is considered.

    if not simon.date().month in (5, 11):
        triggered = check_volatility_trigger(simon, MASTER_DAILY_PRICES_pd, index_rebal_threshold)
        if not triggered:
            continue

    print('------------------------------------')
    print(simon.date())
    print('------------------------------------')

    new_market_cap_data, new_monthly_data, new_weekly_data = build_price_and_mcap_data(nifty100_companies, MASTER_PRICE_STORE, simon - DateOffset(months=36), simon)

    new_momentum_data = initiate_momentum_calculation(simon, new_monthly_data, new_weekly_data)

    new_selected_securities = determine_securities(simon, new_momentum_data, new_market_cap_data, True, prevNumSec)

    curNumSec = len(new_selected_securities.index)

    final_selected_securities = buffer_rule(selected_securities, new_selected_securities, curNumSec)

    momentum_weights = calculate_momentum_weights(new_momentum_data, new_market_cap_data)

    print(f'Top 5 Securities:\n {momentum_weights.head(5)}')

    prevNumSec = curNumSec
    prev_month = simon
    monthly_data, weekly_data, market_cap_data = new_monthly_data, new_weekly_data, new_market_cap_data
    momentum_data = new_momentum_data
    

------------------------------------
2024-11-30
------------------------------------
Top 5 Securities:
         symbol  momentum_score    market_cap  parent_weight  momentum_weight
15  BHARTIARTL        2.778382  9.648396e+12       0.042851         0.101873
40   ICICIBANK        1.553320  9.104013e+12       0.040433         0.053741
76   SUNPHARMA        2.895000  4.262846e+12       0.018932         0.046899
44        INFY        1.646071  7.474044e+12       0.033194         0.046754
34     HCLTECH        2.219907  4.787790e+12       0.021264         0.040391
------------------------------------
2025-05-31
------------------------------------
Top 5 Securities:
         symbol  momentum_score    market_cap  parent_weight  momentum_weight
15  BHARTIARTL        2.774329  1.103596e+13       0.046123         0.108702
41   ICICIBANK        2.167432  1.023259e+13       0.042766         0.078741
36    HDFCBANK        2.426395  7.350428e+12       0.030720         0.063320
71    RELIANCE        